In [1]:
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

MODEL='tabularisai/multilingual-sentiment-analysis'

In [2]:
ds = load_dataset('Yelp/yelp_review_full')
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
# create a 10% random subsample of the original train split
small_subset = ds['train'].train_test_split(train_size=0.1, seed=42)['train']

# split that subsample into 80% train / 20% validation
split = small_subset.train_test_split(test_size=0.2, seed=42)
small_train = split['train']
small_val = split['test']  # validation set

# quick sanity checks
print('original train:', len(ds['train']))
print('subsample (10%):', len(small_subset))
print('small train (80% of subsample):', len(small_train))
print('small val (20% of subsample):', len(small_val))

original train: 650000
subsample (10%): 65000
small train (80% of subsample): 52000
small val (20% of subsample): 13000


In [4]:
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(MODEL)

def hp_space(trial):
    return {
        'learning_rate': trial.suggest_float('learning_rate', 5e-6, 5e-4, log=True),
        'per_device_train_batch_size': trial.suggest_categorical('per_device_train_batch_size', [16, 32]),
        'gradient_accumulation_steps': trial.suggest_categorical('gradient_accumulation_steps', [1, 2, 4]),
    }

def hp_name(trial):
    return f'trial_{trial.number}'

tokenizer = AutoTokenizer.from_pretrained(MODEL)

def tokenize(examples):
    return tokenizer(examples['text'], padding=True, truncation=True, max_length=512)

small_train = small_train.map(tokenize, batched=True, batch_size=512)
small_val = small_val.map(tokenize, batched=True, batch_size=512)
small_train = small_train.remove_columns(['text'])
small_val = small_val.remove_columns(['text'])

training_args = TrainingArguments(
    per_device_eval_batch_size=64,
    metric_for_best_model='accuracy',
    eval_strategy='steps',
    save_strategy='no',
    logging_dir='runs',
    report_to=['tensorboard'],
)

trainer = Trainer(
    model=None,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_val,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
    model_init=model_init,
)

best_trials = trainer.hyperparameter_search(
    direction='maximize',
    backend='optuna',
    hp_space=hp_space,
    hp_name=hp_name,
    n_trials=20,
)

# View best trial results
print("Best trial:")
print(best_trials)

Map:   0%|          | 0/52000 [00:00<?, ? examples/s]

Map:   0%|          | 0/13000 [00:00<?, ? examples/s]

[I 2025-11-29 13:36:52,895] A new study created in memory with name: no-name-1fed67ce-cdff-4a00-8011-c5dabbf34317


Step,Training Loss,Validation Loss,Accuracy
500,1.320900,1.176812,0.481615
1000,1.161400,1.122707,0.514615
1500,1.098400,1.115884,0.522538
2000,1.013900,1.051019,0.538769
2500,0.967100,1.083897,0.527385
3000,0.969700,1.057081,0.547692
3500,0.900900,1.043600,0.559077
4000,0.825100,1.061784,0.551846
4500,0.821600,1.061319,0.558154


[I 2025-11-29 14:13:31,347] Trial 0 finished with value: 0.5581538461538461 and parameters: {'learning_rate': 0.00022290597699294604, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 2}. Best is trial 0 with value: 0.5581538461538461.


...

[I 2025-11-29 20:56:25,665] Trial 18 pruned. 


Step,Training Loss,Validation Loss,Accuracy
500,1.066200,0.984317,0.575846


[I 2025-11-29 21:00:04,272] Trial 19 pruned. 


Best trial:
BestRun(run_id='12', objective=0.6408461538461538, hyperparameters={'learning_rate': 7.26692474158267e-05, 'per_device_train_batch_size': 32, 'gradient_accumulation_steps': 4}, run_summary=None)


AttributeError: 'str' object has no attribute 'objective'